# Schr&ouml;dinger's Equation


### Peter Onyisi
<img src="images/texas_logo.png" width="400" align="left"/>

In [ ]:
# TACC: install if necessary
try:
    import numba
except:
    ! pip3 install --user numba numpy==1.19.1 scipy==1.5.2 tqdm==4.48.2
    INSTALL_PATH='/home/jupyter/mydata/jupyter_packages/lib/python3.6/site-packages'
    import sys, os
    if os.access(INSTALL_PATH, os.R_OK) and INSTALL_PATH not in sys.path:
        sys.path.insert(0, INSTALL_PATH)

# generic
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams['figure.figsize'] = (8,6)
matplotlib.rcParams['font.size'] = 16
matplotlib.rcParams['animation.embed_limit'] = 60
matplotlib.rcParams['xtick.top'] = True
matplotlib.rcParams['ytick.right'] = True
import math
import numpy
import scipy.integrate
import scipy.stats
import scipy.special
from matplotlib import animation

## The Dynamics of Wave Functions

So far we've discussed what wave functions might describe different particle configurations. We haven't talked about how they actually evolve with time - i.e., their *dynamics*. For classical particles, this is handled by Newton's Second Law:

$$ m\frac{d^2 x}{dt^2} = F(x) = -\frac{dV}{dx} $$

in one dimension (where $V$ is the potential energy describing $F$, assuming we're only using conservative forces); in three dimensions we would write

$$ m\frac{d^2\vec{x}}{dt^2} = \vec{F}(\vec{x}) = -\nabla V(\vec{x}) $$

The equivalent for wave functions is a *partial differential equation*, the *time-dependent Schr&ouml;dinger equation*. For one dimension it is:

$$ i\hbar \frac{\partial \psi}{\partial t} = -\frac{\hbar^2}{2m}\frac{\partial^2 \psi}{\partial x^2} + V(x)\psi(x,t) $$

where we have $\hbar = h/2\pi$ and $V(x)$ is the potential energy as a function of $x$. Here we now allow the wave function to be a function of both position $x$ and time $t$: $\psi = \psi(x,t)$.

This is of course a potentially very complicated equation to solve (as are all partial differential equations) but let's try and understand the overall form.  Like Newton's Second Law, it tells us about how a particle will evolve given its state at a certain time.  That is - if $\psi(x, t_0)$ is known, I can evaluate the right hand side of the equation for any value of $x$, and that tells me $\partial \psi/\partial t$ for that $x$.   To make this a little more explicit:

$$ \frac{\partial\psi}{\partial t} = \frac{1}{i\hbar}\left[-\frac{\hbar^2}{2m}\frac{\partial^2 \psi}{\partial x^2} + V(x)\psi(x,t)\right] $$

## Evolution of Momentum States

We can solve the time-dependent Schr&ouml;dinger's equation explicitly in certain cases. Let us look at what happens to a state of definite momentum in a situation where $V(x) = 0$ everywhere, where we specify the constant $C$ at a time $t=0$:

$$ \psi(x, 0) = C \exp(ipx/\hbar) $$

Then, we wind up with the following equation:

$$\begin{align*}
\frac{\partial\psi}{\partial t} &= \frac{1}{i\hbar}\times -\frac{\hbar^2}{2m}\frac{\partial^2}{\partial x^2}\psi \\
&= \frac{i\hbar}{2m} \frac{\partial^2}{\partial x^2} [C \exp(ipx/\hbar)] \\
&= \frac{i\hbar}{2m} \frac{\partial}{\partial x}[C (ip/\hbar)\exp(ipx/\hbar)]\\
&= \frac{i\hbar}{2m} C (ip/\hbar)^2\exp(ipx/\hbar) \\
&= -i \frac{p^2}{2m\hbar} C \exp(ipx/\hbar) \\
&= -i \frac{p^2}{2m\hbar} \psi
\end{align*}$$

which holds for all values of $x$.  From our knowledge of derivatives, we know that the function that has the property that $\partial f/\partial t = K f$ is $f = f(0) \exp(Kt)$ (where $K$ is a constant).  We also know that $p^2/2m = (mv)^2/2m = mv^2/2 $ is the kinetic energy $E$. So we conclude that

$$ \begin{align*}
\psi(x, t) &= \psi(x, 0)\exp(-i\frac{p^2}{2m\hbar}t)\\
&= C\exp(ipx/\hbar)\exp(-iEt/\hbar)\\
&= C\exp(i(px-Et)/\hbar)
\end{align*}
$$

So the solution at all times looks very much like the solution at $t = 0$, except that the wave function is multiplied by $e^{-iEt/\hbar}$.  You will  note this is of the form $e^{i\phi}$, with $\phi$ constant in $x$, so it is just a time-dependent rotation in the complex plane.

## Numerically Solving Partial Differential Equations

In general it is difficult or impossible to find analytic solutions for Schr&ouml;dinger's equation. (You may get the opposite impression in follow-up classes, in which you will spend a lot of time finding analytic solutions! But by definition you will be looking at the cases where analytic solutions are possible.)  Thankfully the time-dependent Schr&ouml;dinger equation is well suited to having a computer approximate the solution.

The general procedure is this:
* since we don't know an analytic form for the wave function, we approximate it by its values at a set of points, assuming it doesn't change too much between the points. In other words we _discretize_ space. We aren't going to be able to see effects that happen at a shorter distance than the spacing between the points, but our computational cost will go up as we have more points, so there is a tradeoff of accuracy for time.
* at any given point in space we can compute $V(x)\psi(x)$ easily. We can _approximate_ $\partial^2 \psi/\partial x^2$ by *finite differences*:
$$ \frac{\partial^2 \psi}{\partial x^2}(x_n) \approx \frac{\psi(x_{n+1})+\psi(x_{n-1})-2\psi(x_n)}{\Delta x^2} $$
* from this information we can determine $\partial \psi/\partial t$. Conceptually we can then use this to extrapolate a small time step into the future, at all grid points:
$$ \psi(x_n, t+\Delta t) \approx \psi(x_n, t) + \Delta t \frac{\partial \psi}{\partial t}(x_n) $$

In practice we make a couple of small modifications:
* Because the finite difference method assumes that there are neighbor grid points on both sides, we have a problem at the edges. We "solve" this by having the grid wrap around, so going off the right edge takes us to the left edge.
* There is a deep literature on how small a time step $\Delta t$ is needed to produce numerically reliable solutions. For the method outlined above, while it works, it requires very small time steps $\Delta t$ to produce good answers. There are more accurate ways to use $\partial\psi/\partial t$ to extrapolate to the future, which rely on effectively computing approximations to $\partial^2 \psi/\partial t^2$ and higher derivatives from $\partial \psi/\partial t$ at different times.  Here we use one of these methods (*Runge-Kutta 5*).

**For all computations here, we set *h*=1, so we don't have to deal with very small/large numbers.**

In [ ]:
RNG=(-10,10)
PTS = 501
grid = numpy.linspace(RNG[0],RNG[1],PTS)[:-1] # drop last point, which wraps to initial point
deltaxsq = ((RNG[1]-RNG[0])/(PTS-1))**2

# define potential energy
V = numpy.zeros_like(grid)
#V = numpy.single(numpy.frompyfunc(lambda x: 20*numpy.sin(x*math.pi/2)**2, 1, 1)(grid))
#V = numpy.single(numpy.frompyfunc(lambda x: 100*x**2, 1, 1)(grid))

# we are going to set h (not hbar) = 1 and m = 1
# so that hbar = 1/(2*pi)

def d2psi(psi):
    return numpy.diff(psi,n=2,prepend=psi[-1],append=psi[0])/deltaxsq

def dpsidt(t, psi):
    global V
    return -1.j*(2*math.pi*V*psi - d2psi(psi)/(2*2*math.pi))
    
P=1
#wave packet
psi=numpy.csingle(scipy.stats.norm(scale=1,loc=-6).pdf(grid))*numpy.exp(2j*math.pi*P*grid)
# normalize psi
psi /= math.sqrt(sum(psi*psi.conj()).real*(RNG[1]-RNG[0])/(PTS-1))

In [ ]:
TRNG=(0,4)
TIMES=numpy.linspace(TRNG[0], TRNG[1], 100)
rk=scipy.integrate.solve_ivp(dpsidt, TRNG, psi, t_eval=TIMES)
print("Done with integration")


In [ ]:
fig, ax = plt.subplots()
liner, = plt.plot([],[],label='$\Re[\psi]$')
linei, = plt.plot([],[],label='$\Im[\psi]$')
linem, = plt.plot([],[],label='$|\psi|^2$')
plt.xlim(*RNG)
plt.ylim(-1,1)
plt.xlabel('$x$')
plt.legend()
def plotter(data):
    global rk
#     ax.clear()
#     ax.axis([RNG[0], RNG[1], 0, 4])

    psii = rk.y[:,data]
#    return (plt.plot(grid, (psii*psii.conj()).real), 
#            plt.plot(grid, V), plt.plot(psi.real))
    liner.set_data(grid, psii.real)
    linei.set_data(grid, psii.imag)
    linem.set_data(grid, (psii*psii.conj()).real)
    return [liner,linei,linem]
    #return plt.plot(grid, psi.real), plt.plot(grid, psi.imag)

print("Rendering...")
ani = animation.FuncAnimation(fig, plotter, len(TIMES))
from IPython.display import HTML
#ani.save('wobble.mp4', writer=animation.writers['ffmpeg'](fps=2))
plt.close()
HTML(ani.to_jshtml())

Let's see if this makes sense. We have set the momentum $p = 1$ in these units, so we would expect the wave packet's velocity $v$ to be $v=p/m$.  If $m = 1$ then we expect $v=1$. Let's see how the peak of $|\psi|^2$ changes with time.

In [ ]:
# Where is the peak of the wavepacket vs time?
psisq = rk.y*rk.y.conj()
maxpos = numpy.argmax(psisq,axis=0)
plt.plot(rk.t, numpy.take_along_axis(grid, maxpos, axis=0))
plt.xlabel('$t$')
plt.ylabel('$x$ of wavepacket peak')
plt.show()

We see that the wavepacket does indeed move with the expected velocity.

Let's do another run with higher momentum, in the opposite direction:

In [ ]:
P=-3
#wave packet
psi=numpy.csingle(scipy.stats.norm(scale=1,loc=6).pdf(grid))*numpy.exp(2j*math.pi*P*grid)
# normalize psi
psi /= math.sqrt(sum(psi*psi.conj()).real*(RNG[1]-RNG[0])/(PTS-1))

In [ ]:
TRNG=(0,4)
TIMES=numpy.linspace(TRNG[0], TRNG[1], 100)
rk=scipy.integrate.solve_ivp(dpsidt, TRNG, psi, t_eval=TIMES)
print("Done with integration")

In [ ]:
fig, ax = plt.subplots()
# liner, = plt.plot([],[],label='$\Re[\psi]$')
# linei, = plt.plot([],[],label='$\Im[\psi]$')
linem, = plt.plot([],[],label='$|\psi|^2$')
plt.xlim(*RNG)
plt.ylim(-1,1)
plt.xlabel('$x$')
plt.legend()
def plotter(data):
    global rk
    psii = rk.y[:,data]
#     liner.set_data(grid, psii.real)
#     linei.set_data(grid, psii.imag)
    linem.set_data(grid, (psii*psii.conj()).real)
#     return [liner,linei,linem]
    return [linem]

print("Rendering...")
ani = animation.FuncAnimation(fig, plotter, len(TIMES))
from IPython.display import HTML
#ani.save('wobble.mp4', writer=animation.writers['ffmpeg'](fps=2))
plt.close()
HTML(ani.to_jshtml())

In [ ]:
# Where is the peak of the wavepacket vs time?
psisq = rk.y*rk.y.conj()
maxpos = numpy.argmax(psisq,axis=0)
plt.plot(rk.t, numpy.take_along_axis(grid, maxpos, axis=0))
plt.xlabel('$t$')
plt.ylabel('$x$ of peak of wavepacket')
plt.show()

What happens if we set P to zero?

In [ ]:
P=0
#wave packet
psi=numpy.csingle(scipy.stats.norm(scale=0.4,loc=0).pdf(grid))*numpy.exp(2j*math.pi*P*grid)
# normalize psi
psi /= math.sqrt(sum(psi*psi.conj()).real*(RNG[1]-RNG[0])/(PTS-1))
TRNG=(0,10)
TIMES=numpy.linspace(TRNG[0], TRNG[1], 100)
rk=scipy.integrate.solve_ivp(dpsidt, TRNG, psi, t_eval=TIMES)
print("Done with integration")
fig, ax = plt.subplots()
# liner, = plt.plot([],[],label='$\Re[\psi]$')
# linei, = plt.plot([],[],label='$\Im[\psi]$')
linem, = plt.plot([],[],label='$|\psi|^2$')
plt.xlim(*RNG)
plt.ylim(-0.5,1.5)
plt.xlabel('$x$')
plt.legend()
def plotter(data):
    global rk
    psii = rk.y[:,data]
#     liner.set_data(grid, psii.real)
#     linei.set_data(grid, psii.imag)
    linem.set_data(grid, (psii*psii.conj()).real)
#     return [liner,linei,linem]
    return [linem]

print("Rendering...")
ani = animation.FuncAnimation(fig, plotter, len(TIMES))
from IPython.display import HTML
plt.close()
HTML(ani.to_jshtml())

What is happening? The _average_ position of the wave packet stays put, but it spreads out. This is because a wave packet is _not_ a pure momentum state, so there are both negative and positive momentum parts of the wave function, and those spread out in opposite directions.

In [ ]:
# Get momentum distribution of wave packet, using a Fourier transform
plt.scatter(numpy.fft.fftfreq(grid.size, d=grid[1]-grid[0]), numpy.abs(numpy.fft.fft(psi)), label='Magnitude of FFT')
plt.xlim(-2,2)
plt.xlabel('$p$')
plt.ylabel('$|\psi(p)|$')
plt.legend()
plt.show()

In [ ]:
# Get momentum distribution of wave packet *after* evolution, using a Fourier transform
plt.scatter(numpy.fft.fftfreq(grid.size, d=grid[1]-grid[0]), numpy.abs(numpy.fft.fft(rk.y[:,-1])), label='Magnitude of FFT')
plt.xlim(-2,2)
plt.xlabel('$p$')
plt.ylabel('$|\psi(p)|$')
plt.legend()
plt.show()

This means that the _magnitude_ of each of the momentum components is actually pretty much the same over time! But how can this be? Well, as always, we need to know how the _phase_ of the Fourier transform components, not just the magnitude.  _That_ changes with time.

In [ ]:
fig = plt.figure()
liner, = plt.plot([],[],label='$\Re[\psi]$')
linei, = plt.plot([],[],label='$\Im[\psi]$')
plt.xlim(*RNG)
plt.xlabel('$p$')
plt.ylim(-30, 30)
plt.xlim(-2,2)
plt.legend()
def plotter(data):
    global rk
    psii = rk.y[:,data]
#     liner.set_data(grid, psii.real)
#     linei.set_data(grid, psii.imag)
    rfft=numpy.fft.fft(psii)
    liner.set_data(numpy.fft.fftfreq(grid.size, d=grid[1]-grid[0]), 
                   rfft.real)
    linei.set_data(numpy.fft.fftfreq(grid.size, d=grid[1]-grid[0]), 
                   rfft.imag)
#     return [liner,linei,linem]
    return [liner,linei]

print("Rendering...")
ani = animation.FuncAnimation(fig, plotter, len(TIMES))
from IPython.display import HTML
plt.close()
HTML(ani.to_jshtml())

It is not usually the case that the magnitude of the momentum components stays constant for a wave function! (It only works here because the potential energy $V(x)$ is a constant.) But in "free space" - i.e. constant potential energy - this is true.

## Non-trivial potentials: Simple Harmonic Oscillator

Let's try something a little more interesting, perhaps. Let's consider a simple harmonic oscillator potential:

$$ V(x) = \frac{1}{2} kx^2 $$

In [ ]:
SPRING_CONSTANT=0.5
V = 0.5*SPRING_CONSTANT*grid**2

In [ ]:
P=0
#wave packet
psi=numpy.csingle(scipy.stats.norm(scale=0.4,loc=1).pdf(grid))*numpy.exp(2j*math.pi*P*grid)
# normalize psi
psi /= math.sqrt(sum(psi*psi.conj()).real*(RNG[1]-RNG[0])/(PTS-1))
TRNG=(0,10)
TIMES=numpy.linspace(TRNG[0], TRNG[1], 100)
rk=scipy.integrate.solve_ivp(dpsidt, TRNG, psi, t_eval=TIMES)
print("Done with integration")
fig, ax = plt.subplots()
# liner, = plt.plot([],[],label='$\Re[\psi]$')
# linei, = plt.plot([],[],label='$\Im[\psi]$')
linem, = plt.plot([],[],label='$|\psi|^2$')
plt.plot(grid, V, label='potential energy $V$')
plt.xlim(*RNG)
plt.ylim(-0.5,1.5)
plt.xlabel('$x$')
plt.legend()
def plotter(data):
    global rk
    psii = rk.y[:,data]
#     liner.set_data(grid, psii.real)
#     linei.set_data(grid, psii.imag)
    linem.set_data(grid, (psii*psii.conj()).real)
#     return [liner,linei,linem]
    return [linem]

print("Rendering...")
ani = animation.FuncAnimation(fig, plotter, len(TIMES))
from IPython.display import HTML
plt.close()
HTML(ani.to_jshtml())

Where is the peak of $|\psi|^2$?

In [ ]:
# Where is the peak of the wavepacket vs time?
psisq = rk.y*rk.y.conj()
maxpos = numpy.argmax(psisq,axis=0)
plt.plot(rk.t, numpy.take_along_axis(grid, maxpos, axis=0))
plt.xlabel('$t$')
plt.ylabel('$x$ position of peak of wavepacket')
plt.show()

From the plot, the motion looks like simple harmonic motion with frequency $\nu \approx$ 1/9. Classically we would expect

$$ \nu = \frac{1}{2\pi}\sqrt{\frac{k}{m}} = 0.1125 \approx 1/9$$

so it seems that the _quantum_ oscillation frequency is the same as the _classical_ one.

What do the momentum magnitudes look like?

In [ ]:
fig = plt.figure()
linem, = plt.plot([],[],label='$|\psi|$')
plt.xlim(*RNG)
plt.ylim(-10, 40)
plt.xlim(-2,2)
plt.xlabel('$p$')
plt.legend()
def plotter(data):
    global rk
    psii = rk.y[:,data]
#     liner.set_data(grid, psii.real)
#     linei.set_data(grid, psii.imag)
    rfft=numpy.fft.fft(psii)
    linem.set_data(numpy.fft.fftfreq(grid.size, d=grid[1]-grid[0]), 
                   numpy.abs(rfft))
#     return [liner,linei,linem]
    return [linem]

print("Rendering...")
ani = animation.FuncAnimation(fig, plotter, len(TIMES))
from IPython.display import HTML
plt.close()
HTML(ani.to_jshtml())

We can imagine that we might be able to get a "stationary state" if we carefully chose the initial form of $\psi$. (The technical definition of this is that $\psi(x, t) = e^{i\phi(t)}\psi(x, t_0)$ for all $t$, i.e. $\psi$ changes only by phase with time.) Playing around gives us a solution:

In [ ]:
psi=numpy.csingle(scipy.stats.norm(scale=0.47,loc=0).pdf(grid))*numpy.exp(2j*math.pi*P*grid)
# normalize psi
psi /= math.sqrt(sum(psi*psi.conj()).real*(RNG[1]-RNG[0])/(PTS-1))
TRNG=(0,10)
TIMES=numpy.linspace(TRNG[0], TRNG[1], 100)
rk=scipy.integrate.solve_ivp(dpsidt, TRNG, psi, t_eval=TIMES)
print("Done with integration")
fig, ax = plt.subplots()
# liner, = plt.plot([],[],label='$\Re[\psi]$')
# linei, = plt.plot([],[],label='$\Im[\psi]$')
linem, = plt.plot([],[],label='$|\psi|^2$')
plt.plot(grid, V, label='potential energy $V$')
plt.xlim(*RNG)
plt.ylim(-0.5,1.5)
plt.xlabel('$x$')
plt.legend()
def plotter(data):
    global rk
    psii = rk.y[:,data]
#     liner.set_data(grid, psii.real)
#     linei.set_data(grid, psii.imag)
    linem.set_data(grid, (psii*psii.conj()).real)
#     return [liner,linei,linem]
    return [linem]

print("Rendering...")
ani = animation.FuncAnimation(fig, plotter, len(TIMES))
from IPython.display import HTML
plt.close()
HTML(ani.to_jshtml())

In fact if one solves for this (and in fact the simple harmonic oscillator is one of the systems where stationary solutions can be solved for exactly) one finds that the _scale_ argument to the normal distribution (i.e., the $\sigma$ parameter) should be 0.4744, which is what we have found by trial and error.

We won't justify the following, but it is possible to estimate the energy of a state by evaluating the term

$$ \frac{1}{\psi}\left[-\frac{\hbar^2}{2m}\frac{\partial^2}{\partial x^2}\psi + V(x)\psi(x,t)\right] $$

If the result is _constant_ in $x$, then the state has a definite energy (and it turns out this means it's a stationary state).

In [ ]:
# Look at the energy of the simple harmonic oscillator
# remember hbar = 1/2pi in our units
# precision falls off where psi ~ 0 ...
psi=numpy.csingle(scipy.stats.norm(scale=0.474425,loc=0).pdf(grid))*numpy.exp(2j*math.pi*P*grid)
# normalize psi
psi /= math.sqrt(sum(psi*psi.conj()).real*(RNG[1]-RNG[0])/(PTS-1))
# fix zero values ...
psi[psi==0]=1e-16
eestimate = ((-d2psi(psi)/(8*math.pi**2)+V*psi)/psi).real
plt.plot(grid, eestimate)
plt.ylim(-0.2,0.2)
plt.xlim(-5,5)
plt.xlabel('$x$')
plt.ylabel(r'$\frac{\hbar^2}{2m}\frac{\partial^2\psi}{\partial x^2} + V\psi$')
print(f'Estimated energy: {numpy.max(eestimate[(grid>-5) & (grid < 5)])}')


There are other possible stationary states, for example:

In [ ]:
psi=numpy.csingle(2*grid*scipy.stats.norm(scale=0.47,loc=0).pdf(grid))*numpy.exp(2j*math.pi*P*grid)
# normalize psi
psi /= math.sqrt(sum(psi*psi.conj()).real*(RNG[1]-RNG[0])/(PTS-1))
TRNG=(0,10)
TIMES=numpy.linspace(TRNG[0], TRNG[1], 100)
rk=scipy.integrate.solve_ivp(dpsidt, TRNG, psi, t_eval=TIMES)
print("Done with integration")
fig, ax = plt.subplots()
# liner, = plt.plot([],[],label='$\Re[\psi]$')
# linei, = plt.plot([],[],label='$\Im[\psi]$')
linem, = plt.plot([],[],label='$|\psi|^2$')
plt.plot(grid, V, label='potential energy $V$')
plt.xlim(*RNG)
plt.ylim(-0.5,1.5)
plt.xlabel('$x$')
plt.legend()
def plotter(data):
    global rk
    psii = rk.y[:,data]
#     liner.set_data(grid, psii.real)
#     linei.set_data(grid, psii.imag)
    linem.set_data(grid, (psii*psii.conj()).real)
#     return [liner,linei,linem]
    return [linem]

print("Rendering...")
ani = animation.FuncAnimation(fig, plotter, len(TIMES))
from IPython.display import HTML
plt.close()
HTML(ani.to_jshtml())

In [ ]:
# Look at the energy of the simple harmonic oscillator
# remember hbar = 1/2pi in our units
# precision falls off where psi ~ 0 ...
psi=numpy.csingle(2*grid*scipy.stats.norm(scale=0.474425,loc=0).pdf(grid))*numpy.exp(2j*math.pi*P*grid)
# normalize psi
psi /= math.sqrt(sum(psi*psi.conj()).real*(RNG[1]-RNG[0])/(PTS-1))
# fix zero values ...
psi[psi==0]=1e-16
eestimate = ((-d2psi(psi)/(8*math.pi**2)+V*psi)/psi).real
plt.plot(grid, eestimate)
plt.ylim(-0.2,0.2)
plt.xlim(-5,5)
plt.xlabel('$x$')
print(f'Estimated energy: {numpy.max(eestimate[(numpy.abs(psi)>1e-5) & (grid>-5) & (grid < 5)])}')

## Simple Harmonic Oscillator and Zero Point Energy

Is this consistent with our simple ideas of quantization?  We expect that any sort of "long-term" state is going to be a stationary state, because otherwise it will evolve quickly.  So we imagine that if we are looking at something that appears stable over time, the particles inside must be in stationary states. Let's note that when we discussed the early idea that simple harmonic oscillators had "minimum energy $h\nu$", this was *the minimum energy above the lowest energy state*. We can *always* redefine the energy of the lowest energy state by changing the potential energy. So what we really want to see is that the *energy difference between adjacent stationary states* is $h\nu$. What do we see here? One state has energy $\approx$ 0.0565, and another state we have considered has energy $\approx$ 0.1691; the difference is 0.1126.  The frequency $\nu$ of this oscillator is 0.1125 (from above), and we fixed $h=1$ for these computations, so indeed we have $\Delta E = h\nu$, to within the accuracy of our numerical computations!

In fact, the full quantum solution for the simple harmonic oscillator gives the following energies for the stationary states:

$$ E_n = \left(n + \frac{1}{2}\right) h\nu $$

so the lowest energy state has $E_0 = h\nu/2$ (the so-called *zero point energy*). Note that we could make this go away by shifting down $V(x)$ by $h\nu/2$, so again the absolute scale is not really meaningful; the main point of the zero point energy is that, given the same potential, where _classical_ physics would allow states with energy $E=0$ (motionless at the bottom of the potential), _quantum_ physics doesn't agree.

The precise reason that quantum mechanics doesn't allow $E=0$ is the following: we saw that a very well-localized wave function contains a giant spread of possible momenta. But non-zero momentum implies non-zero _kinetic_ energy. A state of zero momentum (and so zero kinetic energy) is a constant, hence has non-zero _potential_ energy. Since it is not possible to have a state that simultaneously has zero momentum and fixed position at zero, it is not possible to have total energy zero.

## Quantum Tunneling

In classical physics, the total energy of a particle (the sum of kinetic and potential energy) is well defined, and it is not possible for a particle to have negative kinetic energy. In quantum mechanics, because the wave function is spread out and doesn't have a single kinetic (or potential) energy, it is not as simple a story.

The "kinetic energy" term in Schr&ouml;dinger's equation is

$$ K \sim -\frac{1}{\psi}\frac{\hbar^2}{2m}\frac{\partial^2\psi}{\partial x^2} $$

but there's nothing requiring this to be positive. For _oscillating_ functions $\psi \sim \exp(ipx/\hbar)$, we have 

$$ K \sim +\frac{p^2}{2m} > 0 $$

which is greater than zero. However, for _exponential_ functions $\psi \sim \exp(-kx/\hbar)$, we have instead

$$ K \sim -\frac{k^2}{2m} < 0 $$

So it is possible for a particle being described by a wave function to enter a region where, classically, its kinetic energy would be negative. In such cases the wave function must fall away exponentially the further into this region that you go.  (So the wave function does not vanish immediately when the potential energy gets too high, but rather goes smoothly to zero.)

Now, if we have a tall (but narrow) potential energy barrier, this means that the wave function can _cross_ the barrier. The degree to which is passes through depends on how high and how wide the barrier is.

Let's look at a simulation with a wave packet hitting a barrier. We will set $p = 3$ in our units, so the kinetic energy when $V(x) = 0$ is $p^2/2m = 3^2/2 = 4.5$.

In [ ]:
RNG=(-20,20)
PTS = 1001
grid = numpy.linspace(RNG[0],RNG[1],PTS)[:-1] # drop last point, which wraps to initial point
deltaxsq = ((RNG[1]-RNG[0])/(PTS-1))**2
P=3
#wave packet
psi=numpy.csingle(scipy.stats.norm(scale=0.8,loc=-7).pdf(grid))*numpy.exp(2j*math.pi*P*grid)
# normalize psi
psi /= math.sqrt(sum(psi*psi.conj()).real*(RNG[1]-RNG[0])/(PTS-1))

V = numpy.zeros_like(grid)
V[(grid>-0.5) & (grid<0)]=4

TRNG=(0,8)
TIMES=numpy.linspace(TRNG[0], TRNG[1], 100)
rk=scipy.integrate.solve_ivp(dpsidt, TRNG, psi, t_eval=TIMES)
print("Done with integration")
fig, ax = plt.subplots(figsize=(12,6))
ax2=ax.twinx()
# liner, = plt.plot([],[],label='$\Re[\psi]$')
# linei, = plt.plot([],[],label='$\Im[\psi]$')
linem, = ax.plot([],[],label='$|\psi|^2$')
linev,=ax2.plot(grid, V, color='r')
plt.xlim(*RNG)
plt.xlabel('$x$')
ax.set_ylim(-0.,0.8)
ax2.set_ylim(0,5)
plt.legend([linem,linev], ['$|\psi|^2$', '$V$'])
def plotter(data):
    global rk
    psii = rk.y[:,data]
#     liner.set_data(grid, psii.real)
#     linei.set_data(grid, psii.imag)
    linem.set_data(grid, (psii*psii.conj()).real)
#     return [liner,linei,linem]
    return [linem]

print("Rendering...")
ani = animation.FuncAnimation(fig, plotter, len(TIMES))
from IPython.display import HTML
plt.close()
HTML(ani.to_jshtml())

So it is possible for a particle to hop a barrier that classically would be impossible to cross, and the probability that it does so depends on the height and width of the barrier.  This can be used to make precisely controlled electronic devices to control the flow of electrons in a circuit, and is also required to explain many kinds of reactions (both chemical and nuclear) which need to get through an intermediate state with "too high" an energy.

For example, the reaction $p + p \to {}^2H + e^+ + \nu_e$ in the Sun (or any future fusion reactor) requires the protons to cross the "Coulomb barrier" of the electrostatic repulsion of the two protons before they can be bound together by the strong (but very short-range) nuclear forces. This must be achieved using protons with thermal kinetic energy. Numerially the height of the barrier for proton-proton collisions is $\approx$ 0.5 MeV, while $\frac{3}{2}k_B T \approx 2$ keV for $T=15\times 10^6$ K at the center of the Sun.  Classically, then, it's almost impossible to find protons with high enough thermal energy to fuse; but quantum tunneling through the Coulomb barrier makes it far more likely that the protons will collide.

[![Coulomb barrier](images/coulomb_barrier.png)](https://search.lib.utexas.edu/discovery/fulldisplay?docid=alma991057942374606011&context=L&vid=01UTAU_INST:SEARCH&lang=en&search_scope=MyInst_and_CI&adaptor=Local%20Search%20Engine&tab=Everything&query=any,contains,nuclear%20physics%20of%20stars&offset=0)

## Particle in a Box
One more important case is the *particle in a box*: the particle is in a flat-bottomed potential energy well with "infinitely tall" sides.  This corresponds to a particle trapped in a specific region.

In [ ]:
RNG=(-5,15)
PTS = 1001
grid = numpy.linspace(RNG[0],RNG[1],PTS)[:-1] # drop last point, which wraps to initial point
deltaxsq = ((RNG[1]-RNG[0])/(PTS-1))**2

V = numpy.single(numpy.frompyfunc(lambda x: 1000. if x < 0 or x > 10 else 0, 1, 1)(grid))

psi=numpy.csingle(numpy.frompyfunc(lambda x: math.sin(x/5*8*math.pi) if 0 < x < 10 else 0, 1, 1)(grid))
# normalize psi
psi /= math.sqrt(sum(psi*psi.conj()).real*(RNG[1]-RNG[0])/(PTS-1))
TRNG=(0,1)
TIMES=numpy.linspace(TRNG[0],TRNG[1],50)
rk=scipy.integrate.solve_ivp(dpsidt, TRNG, psi, t_eval=TIMES)
print("Done with integration")

In [ ]:

fig, ax = plt.subplots(figsize=(12,6))
ax2=ax.twinx()
# liner, = plt.plot([],[],label='$\Re[\psi]$')
# linei, = plt.plot([],[],label='$\Im[\psi]$')
linem, = ax.plot([],[],label='$|\psi|^2$')
linev,=ax2.plot(grid, V, color='r')
plt.xlim(*RNG)
plt.xlabel('$x$')
ax.set_ylim(-0.,0.8)
ax2.set_ylim(0,5)
plt.legend([linem,linev], ['$|\psi|^2$', '$V$'])
def plotter(data):
    global rk
    psii = rk.y[:,data]
#     liner.set_data(grid, psii.real)
#     linei.set_data(grid, psii.imag)
    linem.set_data(grid, (psii*psii.conj()).real)
#     return [liner,linei,linem]
    return [linem]

print("Rendering...")
ani = animation.FuncAnimation(fig, plotter, len(TIMES))
from IPython.display import HTML
plt.close()
HTML(ani.to_jshtml())

For a *stationary solution*, we want to have $\psi(x)$ go to zero at the walls of the box (the particle should never be found outside the box (i.e. in the "inifinte potential" region), and the wave function is continuous). Inside the box, since the potential is constant and zero, we might expect an oscillatory solution to work (like in our first example); however $\exp(ipx/\hbar)$ isn't zero anywhere.  However, we see that if $\psi_1$ and $\psi_2$ are solutions of Schr&ouml;dinger's equation, then so is $A\psi_1 + B\psi_2$, because the equation is _linear_ in all the derivatives. So we can add together arbitrary solutions to get another solution. In particular, if we add together right- and left-headed waves, for example:

$$\begin{align*}
\psi(x) &= C \exp(ipx/\hbar) + C\exp(i(-p)x/\hbar) \\
&= C[\cos(px/\hbar) + i\sin(px/\hbar) + \cos(-px/\hbar) + i\sin(-px/\hbar)] \\
&= C[\cos(px/\hbar) + i\sin(px/\hbar) + \cos(px/\hbar) - i\sin(px/\hbar)] \\
&= 2C\cos(px/\hbar)
\end{align*}
$$

we can get a wave function that is zero at various locations. (Note that here, $p$ does _not_ indicate the momentum, since we've combined two wave functions of different momenta! Also, since the wave functions don't go off to infinity, the momentum is not sharply defined anyway.)

In [ ]:
plt.scatter(numpy.fft.fftfreq(grid.size, d=grid[1]-grid[0]), numpy.fft.fft(psi).real, label='FFT Real')
plt.scatter(numpy.fft.fftfreq(grid.size, d=grid[1]-grid[0]), numpy.fft.fft(psi).imag, label='FFT Imag')
plt.xlim(-2,2)
plt.xlabel('$p$')
plt.legend()
plt.show()

We usually set up the problem so that the bottom of the well goes from $0$ to $L$; then what we want are the sine solutions:

$$\begin{align*}
\psi(x) &= \frac{1}{i}[C \exp(ipx/\hbar) - C\exp(i(-p)x/\hbar)] \\
&= \frac{1}{i}C[\cos(px/\hbar) + i\sin(px/\hbar) - \cos(-px/\hbar) - i\sin(-px/\hbar)] \\
&= \frac{1}{i}C[\cos(px/\hbar) + i\sin(px/\hbar) - \cos(px/\hbar) + i\sin(px/\hbar)] \\
&= 2C\sin(px/\hbar)
\end{align*}
$$

$\psi(0)=0$ naturally here, and the condition $\psi(L) = 0$ forces

$$ \frac{pL}{\hbar} = n\pi $$
$$ p = \frac{n\pi\hbar}{L} $$

So the solutions are

$$\psi_n(x) = \sqrt{\frac{2}{L}} \sin\left[\frac{n\pi}{L}x\right] $$

just as in a usual "standing wave" (the factor $\sqrt{2/L}$ is introduced to make the integral of $|\psi|^2$ equal to one.)  The total energies of the states turn out to be

$$ E_n = \frac{n^2\pi^2\hbar^2}{2mL^2} $$

We're interested in the particle in a box as it can provide a rough model for how particles behave when they are "confined" in a region. So, for example, electrons in a metal can to a certain extent be modeled this way: they are free to move around the metal (it's a conductor), but it is very difficult for them to escape the metal (due to the work function).

Note that we can add any number of solutions $\psi_n$ derived above to get a new valid solution; but all valid solutions can be expressed by summing together the $\psi_n$.  (This property means that the $\psi_n$ form what we call a _basis_ for the solutions.)

## Higher dimension example
We can also write Schr&ouml;dinger's equation in more than one dimension. The general equation is

$$ i\hbar \frac{\partial \psi}{\partial t} = -\frac{\hbar^2}{2m}\nabla^2 \psi + V(x)\psi(x,t) $$

so, for example, in 2 dimensions

$$ i\hbar \frac{\partial \psi}{\partial t} = -\frac{\hbar^2}{2m}\left[\frac{\partial^2}{\partial x^2} + \frac{\partial^2}{\partial y^2}\right] \psi + V(x,y)\psi(x,y,t) $$

Let's look at a 2D example: a particle passing down a tunnel which encounters two open slits.  The walls are modeled by making a very high potential barrier.

In [ ]:
RNG_X=(-10,10)
RNG_Y=(-5,5)
PTS_X = 401
PTS_Y = 61
gridvalx = numpy.linspace(RNG_X[0],RNG_X[1],PTS_X)[:-1] # drop last point, which wraps to initial point
gridvaly = numpy.linspace(RNG_Y[0],RNG_Y[1],PTS_Y)[:-1] # drop last point, which wraps to initial point
gridx, gridy = numpy.meshgrid(gridvalx, gridvaly)

deltaxsq = ((RNG_X[1]-RNG_X[0])/(PTS_X-1))**2
deltaysq = ((RNG_Y[1]-RNG_Y[0])/(PTS_Y-1))**2

def d2psi(psi):
    psiloc = psi.reshape(gridx.shape)
    derivy=numpy.diff(psiloc,n=2,prepend=psiloc[-1:,:],append=psiloc[:1,:], axis=0)/deltaysq
    derivx=numpy.diff(psiloc,n=2,prepend=psiloc[:,-1:],append=psiloc[:,:1], axis=1)/deltaxsq
    return derivx+derivy

def dpsidt(t, psi):
    global V
    return (-1.j*(2*math.pi*V*psi.reshape(gridx.shape) - d2psi(psi)/(4*math.pi))).reshape(-1)

# double slit with incident wave
SLITWIDTH=0.6
SLITCENTER=0.6
ORIGWIDTH=4.
V = numpy.single(numpy.frompyfunc(lambda x, y: 0 if (SLITCENTER-SLITWIDTH/2 < abs(y) < SLITCENTER+SLITWIDTH/2 and x < 0) 
                                  or x >= 0 else 20, 2, 1)(gridx, gridy))
V[(numpy.abs(gridy)<ORIGWIDTH/2) & (gridx<-0.3)]=0
    
P = 2
psi=numpy.csingle(scipy.stats.norm(scale=0.5,loc=-2).pdf(gridx))*numpy.exp(2j*math.pi*P*gridx)*numpy.cos(gridy*math.pi/2./(ORIGWIDTH/2))
psi[(numpy.abs(gridy) > ORIGWIDTH/2)]=0
psi[gridx>=-0.3]=0
# normalize psi
psi /= math.sqrt(numpy.sum(psi*psi.conj()).real*(RNG_X[1]-RNG_X[0])*(RNG_Y[1]-RNG_Y[0])/(PTS_X-1)/(PTS_Y-1))

TRNG=(0,4)
NTIMES=50
rk=scipy.integrate.solve_ivp(dpsidt, TRNG, psi.reshape(-1), t_eval=numpy.linspace(TRNG[0],TRNG[1],NTIMES))
psi = psi.reshape(gridx.shape)
print("Done with integration")



In [ ]:
fig, ax = plt.subplots()
ax.set_ylim(*RNG_Y)
plt.xlabel('$x$')
plt.ylabel('$y$')
ax.contourf(gridx, gridy, V,vmin=1,vmax=9, cmap=plt.cm.bone.reversed())
psicont = ax.contourf(gridx,gridy,rk.y[:,0].real.reshape(psi.shape))
def plotter(data):
    global rk, psicont
    for c in psicont.collections:
        c.remove()
    psii = rk.y[:,data].reshape(psi.shape)
    psicont = ax.contour(gridx, gridy, (psii*psii.conj()).real,levels=numpy.linspace(0.,0.5,101))
    return [psicont]

print("Rendering...")
ani = animation.FuncAnimation(fig, plotter, NTIMES)
from IPython.display import HTML
plt.close()
HTML(ani.to_jshtml())

As one might expect, we get a diffraction pattern. 

In [ ]:
# at last timeslice, look at |psi|^2 vs y at x=5
# find x bin corresponding to x=5
psislice=rk.y[:,-1].reshape(psi.shape)[gridx==5]
plt.plot(gridy[gridx==5],(psislice*psislice.conj()).real)
plt.xlabel('$y$')
plt.ylabel('$|\psi(5,y)|^2$')
plt.show()

What does passage through a single slit look like?

In [ ]:
V = numpy.single(numpy.frompyfunc(lambda x, y: 0 if (SLITCENTER-SLITWIDTH/2 < y < SLITCENTER+SLITWIDTH/2 and x < 0) 
                                  or x >= 0 else 20, 2, 1)(gridx, gridy))
V[(numpy.abs(gridy)<ORIGWIDTH/2) & (gridx<-0.3)]=0
    
P = 2
psi=numpy.csingle(scipy.stats.norm(scale=0.5,loc=-2).pdf(gridx))*numpy.exp(2j*math.pi*P*gridx)*numpy.cos(gridy*math.pi/2./(ORIGWIDTH/2))
psi[(numpy.abs(gridy) > ORIGWIDTH/2)]=0
psi[gridx>=-0.3]=0
# normalize psi
psi /= math.sqrt(numpy.sum(psi*psi.conj()).real*(RNG_X[1]-RNG_X[0])*(RNG_Y[1]-RNG_Y[0])/(PTS_X-1)/(PTS_Y-1))

TRNG=(0,4)
NTIMES=50
rk=scipy.integrate.solve_ivp(dpsidt, TRNG, psi.reshape(-1), t_eval=numpy.linspace(TRNG[0],TRNG[1],NTIMES))
psi = psi.reshape(gridx.shape)
print("Done with integration")

In [ ]:
fig, ax = plt.subplots()
ax.set_ylim(*RNG_Y)
plt.xlabel('$x$')
plt.ylabel('$y$')
ax.contourf(gridx, gridy, V,vmin=1,vmax=9, cmap=plt.cm.bone.reversed())
psicont = ax.contourf(gridx,gridy,rk.y[:,0].real.reshape(psi.shape))
def plotter(data):
    global rk, psicont
    for c in psicont.collections:
        c.remove()
    psii = rk.y[:,data].reshape(psi.shape)
    psicont = ax.contour(gridx, gridy, (psii*psii.conj()).real,levels=numpy.linspace(0.,0.5,101))
    return [psicont]

print("Rendering...")
ani = animation.FuncAnimation(fig, plotter, NTIMES)
from IPython.display import HTML
plt.close()
HTML(ani.to_jshtml())

## Particle in a 3D box

In three dimensions, Schr&ouml;dinger's equation is

$$ i\hbar \frac{\partial \psi}{\partial t} = -\frac{\hbar^2}{2m}\left[\frac{\partial^2}{\partial x^2} + \frac{\partial^2}{\partial y^2} + \frac{\partial^2}{\partial z^2}\right] \psi + V(x,y,z)\psi(x,y,z,t) $$

We can set up a cuboidal box with sides $[L_x, L_y, L_z]$ similar to the earlier example, with $V(x)=0$ inside the box and "infinity" outside. It turns out that in this case the differential equation is called _separable_ and we can assume the solutions are products of the solutions for each dimension, i.e.

$$\psi_{(n_x,n_y,n_z)}(x) = \sqrt{\frac{2}{L_x}} \sqrt{\frac{2}{L_y}} \sqrt{\frac{2}{L_z}}\sin\left[\frac{n_x\pi}{L_x}x\right]\sin\left[\frac{n_y\pi}{L_y}y\right]\sin\left[\frac{n_z\pi}{L_z}z\right] $$

where $n_x$, $n_y$, $n_z$ are positive integers. The energies of the states are

$$ E_{(n_x,n_y,n_z)} = \frac{\pi^2\hbar^2}{2m}\left(\frac{n_x^2}{L_x^2}+\frac{n_y^2}{L_y^2}+\frac{n_z^2}{L_z^2}\right) $$

If the box is a cube of side length $L$, this is just

$$ E_{(n_x,n_y,n_z)} = \frac{\pi^2\hbar^2}{2m}\left(\frac{n_x^2+n_y^2+n_z^2}{L^2}\right) $$

A thing that we will at some point want to know is how many possible basis states $\psi_{(n_x,n_y,n_z)}$ are to be found with energy less than a certain energy $E$.  If we look at a cube of side length $L$, then there is an acceptable state at every lattice point $(n_x, n_y, n_z)$ satisfying

$$ n_x^2+n_y^2+n_z^2 < \frac{2mL^2 E}{\pi^2\hbar^2}$$

(note that $n_x$, $n_y$, $n_z$ are positive integers).  But the number of such lattice points is approximately the same as $1/8$ times the volume of the sphere of radius $\sqrt{n_x^2+n_y^2+n_z^2}$. So we expect that the number of states $N$ with energy $\le E$ is

$$ \begin{align*}
N(E) &= \frac{1}{8} \times \frac{4}{3} \pi r^3\\
&=\frac{1}{6} \pi \left[\frac{2mL^2 E}{\pi^2\hbar^2}\right]^{3/2}\\
&=\frac{1}{6} \frac{(2m)^{3/2}L^3}{\pi^2\hbar^3} E^{3/2} = \frac{4\pi(2m)^{3/2}}{3}\frac{L^3}{h^3} E^{3/2}
\end{align*}
$$

Note that this result is proportional to the volume $L^3$ of the cube, and includes a $1/h^3$ factor (like we had when we talked about counting states in statistical mechanics).  We can take this particle in a box model as a model for the ideal gas (in terms of state counting) and derive the Sackur-Tetrode equation this way.